In [1]:
import requests
import time
import os
import shutil
import subprocess
import concurrent.futures
import pandas as pd
from dotenv import load_dotenv
load_dotenv("../../.env")

True

In [6]:
# General Define
heygen_api_key  = os.getenv("HEYGEN_API_KEY")
headers         = {"Accept": "application/json", "X-API-KEY": heygen_api_key}
template_id     = os.getenv("TEMPLATE_ID")

generate_url    = f"https://api.heygen.com/v2/template/{template_id}/generate"

In [3]:
#Payload
def payload_setting(title, script):
    
    # payload = {
    #     "test": True,
    #     "caption": False,
    #     "title": title,
    #     "dimension": {
    #             "width": 1920,  # Fix
    #             "height": 1080  # fix
    #         },
    #     "variables": {
    #         "script": {
    #             "name": "script",
    #             "type": "text",
    #             "properties": {"content": script},
    #         }
    #     },
    # }
    
    payload = {
        "caption": False,  # Caption, maybe the subtile
        "title": title,  # Need to change , match the index of video
        "callback_id": "string",
        "dimension": {
            "width": 1024,  # Fix
            "height": 720  # fix
        },
        "video_inputs": [      # up to 50, don't know whether it is video setting or many video at a time
            {
                "character": {
                    "type": "avatar",
                    "avatar_id": "Angela-inTshirt-20220820",
                    "avatar_style": "normal"
                },
                "voice": {
                    "type": "text",
                    "voice_id": "c6fb81520dcd42e0a02be231046a8639", #str
                    "input_text": script #str
                },   # voice id Fixx
            }
        ],
        "callback_url": "string"
    }
    
    return payload

In [7]:
payload = payload_setting("1", "this is on the fly")

headers["Content-Type"] = "application/json"
generate_url = "https://api.heygen.com/v2/video/generate"
response = requests.post(generate_url, headers=headers, json=payload)
if not response.json()["data"]:
    print(response)
    print(response.json()["error"])
    
    

video_id = response.json()["data"]["video_id"]
print("video_id:", video_id)

video_id: bcc0623798634f1d919660636bbd7ba4


In [8]:
payload = payload_setting("2", "this is on the fly")

headers["Content-Type"] = "application/json"
generate_url = "https://api.heygen.com/v2/video/generate"
response = requests.post(generate_url, headers=headers, json=payload)
if not response.json()["data"]:
    print(response)
    print(response.json()["error"])
    
    

video_id = response.json()["data"]["video_id"]
print("video_id:", video_id)

video_id: 37cfa8feb8d645fda8446865e4b30742


In [16]:
payload = payload_setting("10", "this is on the fly")

headers["Content-Type"] = "application/json"
generate_url = "https://api.heygen.com/v2/video/generate"
response = requests.post(generate_url, headers=headers, json=payload)
if not response.json()["data"]:
    print(response)
    print(response.json()["error"])
    
    

video_id = response.json()["data"]["video_id"]
print("video_id:", video_id)

video_id: ba1c05fe65ed4e0db5530539685fb43e


In [17]:
import time

start_time = time.time()
for i in range(10):
    print("ccc")
    time.sleep(3)
    
print(time.time() - start_time )

ccc
ccc
ccc
ccc
ccc
ccc
ccc
ccc
ccc
ccc
30.022815465927124


In [29]:
def video_generator(index, title, script):
    
    payload = payload_setting(title, script)
    
    headers["Content-Type"] = "application/json"
    response = requests.post(generate_url, headers=headers, json=payload)
    if not response.json()["data"]:
        print(response)
        print(response.json()["error"])
        
        return index, title, "error", ""
        

    video_id = response.json()["data"]["video_id"]
    print("video_id:", video_id)
    
    
    # Check Video Generation Status
    video_status_url = f"https://api.heygen.com/v1/video_status.get?video_id={video_id}"
    while True:
        response = requests.get(video_status_url, headers=headers)
        status = response.json()["data"]["status"]

        if status == "completed":
            video_url = response.json()["data"]["video_url"]
            thumbnail_url = response.json()["data"]["thumbnail_url"]
            print(
                f"Video generation completed! \nVideo URL: {video_url} \nThumbnail URL: {thumbnail_url}"
            )

            # Save the video to a file
            video_filename = f"../../data/videos/{title}.mp4"
            with open(video_filename, "wb") as video_file:
                video_content = requests.get(video_url).content
                video_file.write(video_content)
                
            return index, title, status, video_url

        elif status == "processing" or status == "pending":
            print("Video is still processing. Checking status...")
            time.sleep(5)  # Sleep for 5 seconds before checking again
        elif status == "failed":
            error = response.json()["data"]["error"]
            print(f"Video generation failed. '{error}'")
            
            return index, title, status, ""
            

    
    

In [30]:
qa_df = pd.read_excel("../../data/question_answer/QA_heygen.xlsx")

In [31]:
qa_df["video_id"]       = ""
qa_df["video_id"]       = qa_df["video_id"].astype(str)

qa_df["video_status"]   = ""
qa_df["video_status"]   = qa_df["video_status"].astype(str)

qa_df["video_url"]      = ""
qa_df["video_url"]      = qa_df["video_url"].astype(str)

qa_df.to_excel("../../data/question_answer/QA_heygen_init_result.xlsx", index=False)


In [32]:



# Run in parallel with ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:  # Adjust workers as needed
    for id, row in qa_df.iterrows():
        
        title   = row["no"]
        script  = row["answer"]
        
        futures = {executor.submit(video_generator, id, title, script)}
        
    for future in concurrent.futures.as_completed(futures):
        idx, video_id, video_status, video_url = future.result()

        qa_df.at[idx, "video_id"] = video_id
        qa_df.at[idx, "video_status"] = video_status
        qa_df.at[idx, "video_url"] = video_url

# Save the updated DataFrame to a new Excel file
qa_df.to_excel("../../data/question_answer/QA_heygen_final_result.xlsx", index=False)

video_id: 8734f55dec194c239bc0522a979a4ce7
video_id: 6aee45ae38c845abb73718beab4476a4
Video is still processing. Checking status...
Video generation failed. '{'code': 401029, 'detail': 'Please subscribe to higher plan to generate higher resolution videos', 'message': 'Resolution not allowed'}'
Video generation failed. '{'code': 401029, 'detail': 'Please subscribe to higher plan to generate higher resolution videos', 'message': 'Resolution not allowed'}'


In [4]:
import requests

url = "http://172.17.16.10:8006/generate_video"

data = {
    "no": "10203838",
    "question": "What is FastAPI?",
    "answer": "FastAPI is a modern"
}

response = requests.post(url, json=data)

print("Status Code:", response.status_code)
print("Response JSON:", response.json())

Status Code: 200
Response JSON: {'video_url': 'https://files2.heygen.ai/aws_pacific/avatar_tmp/f45bd8b98c644eecad322ef3225c78bf/af9e0aafbed646a88eb1d5d78b4df7a9.mp4?Expires=1741763602&Signature=jkc47pKKP3pVGD8ZH2owG5S-iWCcRe6LpSOZQB5kkArnqEG41-KlWerDMc7PeORftIoGovJbH0Xdv7o6s3qF-Xxp5aWNNVm~nrlrXH6PhQU2NIByZhA1QPnlIqISb9aFpWZubT2oux2BiXtmtAJBgU8wtINpo-YEg5mjp9-tdfVMrTIEdecLRISMWaZZ09eTy9-Q7kQgWPsHjhBYBj~c6XkMy8q2YZ8afR-Cl95zlKNDrBSWcmUvXiZZ3Ae5X629equJjjZG0jcdn~xZckIbZoF0E03QqOnCHM7pk1aKxW7oXFYKiUJKjnSuhSxx-cjt7zZthYp0wmraqkPOfwIY7w__&Key-Pair-Id=K38HBHX5LX3X2H'}
